# Get required imports

In [1]:
import multiprocessing
import socket
import random

# First Layer the User Datagram Protocol.
    - Constructor: The user assigns a channel to bind with the server which will send the informaton through.
    - sendMessage(): Sends through the channel a message to the server.
    - recieveMessage(): Listens and recieves any replies to confirm the message arrived.
    - closeConnection(): Shuts down the server.

In [2]:
class UDPClient:

    def __init__(self, server_address, debug):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.server_address = server_address
        self.debug = debug

    def sendMessage(self, message):
        if random.randint(0, 5) == 0:
            if self.debug:
                print('\nDEBUG - Data did not send!')
            return False
        else:
            if self.debug:
                print('\nDEBUG - Sending {!r}'.format(message))
            if random.randint(0, 10) == 0:
                if self.debug:
                    print('DEBUG - Sent data twice!')
                self.sock.sendto(str.encode(message), self.server_address)
            self.sock.sendto(str.encode(message), self.server_address)
            return True

    def recieveMessage(self):
        if self.debug:
            print('DEBUG - waiting to receive')
        data, server = self.sock.recvfrom(4096)
        if self.debug:
            print('DEBUG - received {!r}'.format(data))
        return data.decode()

    def closeConnection(self):
        if self.debug:
            print('DEBUG - closing socket')
        self.sock.close()
        return True

# Second Layer the Stop and Wait Protocol.
    - Constructor: The user assigns a channel to send messages to the server.
    - sendMessage(): Sends messeges and waits and recieves any replies that arrives in the channel. If we dont recieve any or recieve a wrong one then we send it again.
    - closeConnection(): Shuts down the server.

In [3]:
class SAWClient:

    def __init__(self, ip_address, debug):
        self.conn = UDPClient((ip_address, 5000), debug)
        self.debug = debug
        self.index = 0
        self.indexLimit = 20
        self.divider = '!#$'
        self.timeOutTime = 3
        self.memoryQueue = list()
        for i in range(0, 10):
            self.memoryQueue.append('')

    def sendMessage(self, message):
        frame = str(self.index) + self.divider + message
        replied = False
        timeOutLimit = 10
        while not replied:
            self.conn.sendMessage(frame)
            try:
                pool = multiprocessing.Pool(processes=1)
                result = pool.apply_async(self.conn.recieveMessage, ())
                acknowledge = result.get(timeout=self.timeOutTime)
                if self.debug:
                    print('DEBUG - Reply: ' + acknowledge)
                if acknowledge == str(self.index):
                    self.memoryQueue.insert(0, acknowledge)
                    self.memoryQueue.pop()
                    replied = True
                    self.index = (self.index + 1) % self.indexLimit
                    if self.debug:
                        print("DEBUG - Acknowledgement Received")

            except multiprocessing.context.TimeoutError:
                if self.debug:
                    print('DEBUG - Acknowledgement Not Received, Timed Out')
                timeOutLimit = timeOutLimit - 1
                if timeOutLimit == 0:
                    if self.debug:
                        print('ERROR - Connection Lost')
                    return False

        return True

    def closeConnection(self):
        self.conn.closeConnection()


# Client Interface.
    - Starts by asking the user for a message to send and calls the proper functions to make that happen.

In [ ]:
exit = False
client = SAWClient('127.0.0.1', True)

while not exit:
    message = input("\nSend message:")
    print('')
    success = client.sendMessage(message)
    if message == "exit" or not success:
        exit = True

client.closeConnection()


Send message:Hi


DEBUG - Sending '0!#$Hi'
DEBUG - waiting to receive
DEBUG - received b'0'
DEBUG - Reply: 0
DEBUG - Acknowledgement Received

Send message:Hello


DEBUG - Data did not send!
DEBUG - waiting to receive
DEBUG - received b'0'
DEBUG - Reply: 0

DEBUG - Sending '1!#$Hello'
DEBUG - waiting to receive
DEBUG - received b'1'
DEBUG - Reply: 1
DEBUG - Acknowledgement Received

Send message:Yo


DEBUG - Sending '2!#$Yo'
DEBUG - waiting to receive
DEBUG - received b'2'
DEBUG - Reply: 2
DEBUG - Acknowledgement Received

Send message:Bastard


DEBUG - Data did not send!
DEBUG - waiting to receive
DEBUG - Acknowledgement Not Received, Timed Out

DEBUG - Sending '3!#$Bastard'
DEBUG - received b'3'
DEBUG - waiting to receive
DEBUG - Acknowledgement Not Received, Timed Out

DEBUG - Sending '3!#$Bastard'
DEBUG - received b'3'
DEBUG - waiting to receive
DEBUG - Acknowledgement Not Received, Timed Out

DEBUG - Sending '3!#$Bastard'
DEBUG - received b'3'
DEBUG - waiting to receive
DEBUG - A